In [23]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import math
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pickle

In [24]:
loan_data_test = pd.read_csv('../datasets/bankrecords.csv')

In [25]:
loan_data_test

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0.0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3.0,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1.0,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2.0,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [26]:
del loan_data_test['Loan_ID']

In [27]:
loan_data_test.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [28]:
x_ins = loan_data_test.drop(columns =['Loan_Status'])
y_ins = loan_data_test['Loan_Status']

In [29]:
x_ins

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban
1,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural
2,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban
3,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban
4,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0.0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural
610,Male,Yes,3.0,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural
611,Male,Yes,1.0,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban
612,Male,Yes,2.0,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban


In [30]:
numerical_cols = loan_data_test.select_dtypes(include=np.number).columns.tolist()
categorical_cols = loan_data_test.select_dtypes('object').columns.tolist()

In [31]:
numerical_cols

['Dependents',
 'ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History']

In [32]:
categorical_cols

['Gender',
 'Married',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Loan_Status']

In [33]:
impute_ins = SimpleImputer(strategy='mean').fit(loan_data_test[numerical_cols])
impute_ins.transform(loan_data_test[numerical_cols])

array([[0.00000000e+00, 5.84900000e+03, 0.00000000e+00, 1.46412162e+02,
        3.60000000e+02, 1.00000000e+00],
       [1.00000000e+00, 4.58300000e+03, 1.50800000e+03, 1.28000000e+02,
        3.60000000e+02, 1.00000000e+00],
       [0.00000000e+00, 3.00000000e+03, 0.00000000e+00, 6.60000000e+01,
        3.60000000e+02, 1.00000000e+00],
       ...,
       [1.00000000e+00, 8.07200000e+03, 2.40000000e+02, 2.53000000e+02,
        3.60000000e+02, 1.00000000e+00],
       [2.00000000e+00, 7.58300000e+03, 0.00000000e+00, 1.87000000e+02,
        3.60000000e+02, 1.00000000e+00],
       [0.00000000e+00, 4.58300000e+03, 0.00000000e+00, 1.33000000e+02,
        3.60000000e+02, 0.00000000e+00]])

In [34]:
encode_ins = OneHotEncoder(sparse=False, handle_unknown='ignore').fit(loan_data_test[categorical_cols])
cols_encode_ins = list(encode_ins.get_feature_names(categorical_cols))
loan_data_test[cols_encode_ins] = encode_ins.transform(loan_data_test[categorical_cols])

In [35]:
loan_data_test_inst = loan_data_test[numerical_cols + cols_encode_ins]

In [38]:
loan_data_test_inst.columns

Index(['Dependents', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Gender_Female', 'Gender_Male',
       'Gender_nan', 'Married_No', 'Married_Yes', 'Married_nan',
       'Education_Graduate', 'Education_Not Graduate', 'Self_Employed_No',
       'Self_Employed_Yes', 'Self_Employed_nan', 'Property_Area_Rural',
       'Property_Area_Semiurban', 'Property_Area_Urban', 'Loan_Status_N',
       'Loan_Status_Y'],
      dtype='object')